<a href="https://colab.research.google.com/github/gfever/ComfyCloud_My_Work_Flow/blob/main/ComfyCloud_My_Work_Flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ComfyCloud v0.1.2

**ComfyUI + ComfyUI Manager + AnimateDiff (Evolved)**

Forked from [ltdrdata/ComfyUI-Manager](https://colab.research.google.com/github/ltdrdata/ComfyUI-Manager/blob/main/notebooks/comfyui_colab_with_manager.ipynb), modified by [@dreammachineai](https://instagram.com/dreammachineai)<br>
modified by [@WilliamJiamin](https://youtube.com/@learn-digital-art)

## Notes

- Uncheck `INSTALL_COMFYUI_MANAGER`, `INSTALL_ANIMATEDIFF`, and `INSTALL_CUSTOM_NODES_DEPENDENCIES` after first run if `USE_GOOGLE_DRIVE` is checked.

I Recommend just check everything, because it will auto skip.

- Model, VAE, Controlnet et al. resources saved to Google Drive if `USE_GOOGLE_DRIVE` is checked
- Resources downloaded from https://civitai.com/ need the file name and extension specified in `resource_name`

## Install Dependencies

In [ ]:
# #@title Environment Setup

from pathlib import Path

OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
INSTALL_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_ANIMATEDIFF = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['INSTALL_COMFYUI_MANAGER'] = INSTALL_COMFYUI_MANAGER
OPTIONS['INSTALL_ANIMATEDIFF'] = INSTALL_ANIMATEDIFF
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

current_dir = !pwd
AI_WORKSPACE = f"{current_dir[0]}/AI"
COMFY_WORKSPACE = f"{current_dir[0]}/AI/ComfyUI"

if OPTIONS['USE_GOOGLE_DRIVE']:
  !echo "Mounting Google Drive..."
  %cd /

  from google.colab import drive
  drive.mount('/content/drive')

  AI_WORKSPACE = "/content/drive/MyDrive/AI"
  COMFY_WORKSPACE = "/content/drive/MyDrive/AI/ComfyUI"

import os
!echo "Creating AI workspace (if it doesn't exist)"
os.makedirs(AI_WORKSPACE, exist_ok=True)

%cd $AI_WORKSPACE

![ ! -d $COMFY_WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $COMFY_WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo "-= Updating ComfyUI =-"
  !git pull

!echo "-= Install dependencies =-"
!pip install -q --no-cache-dir -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121
!pip install -q --no-cache-dir onnxruntime-gpu color-matcher simpleeval torchsde av spandrel kornia
!echo "Dependencies installed."

if OPTIONS['INSTALL_COMFYUI_MANAGER']:
  %cd custom_nodes
  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull

if OPTIONS['INSTALL_ANIMATEDIFF']:
  %cd ../
  ![ ! -d ComfyUI-AnimateDiff-Evolved ] && echo -= Initial setup AnimateDiff =- && git clone https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved
  %cd ComfyUI-AnimateDiff-Evolved
  !git pull

%cd $COMFY_WORKSPACE

if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
  !pwd
  !echo "-= Install custom nodes dependencies =-"
  ![ -f "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py" ] && python "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py"


## Download resources (models, VAEs, LoRAs, Controlnets et al.)

In [ ]:
#@markdown ###Download custom resource (OPTIONAL)
resource_url = "https://civitai.com/api/download/models/222710?type=Model&format=SafeTensor&size=pruned&fp=fp16" #@param {type:"string"}
#@markdown (NOTE: Specify file name and extension for civitai resources)
resource_name = "pixel_art_fastModeExperimental.safetensors" #@param {type:"string"}
output_path = "./models/checkpoints/" # @param ["./models/checkpoints/", "./models/vae/", "./models/loras/", "./models/controlnet/", "./models/style_models/", "./models/upscale_models/", "./custom_nodes/ComfyUI-AnimateDiff-Evolved/models/", "./custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora/", "./models/ipadapter/"]

if resource_url and output_path:
  if resource_name:
    !wget -c "{resource_url}" -O "{output_path}{resource_name}"
  else:
    !wget -c "{resource_url}" -P "{output_path}"

In [ ]:
#@markdown ###Download standard resources
#@markdown (NOTE: These resources are saved if "Using Google Drive" is checked, so you only need to run once).
### SDXL
### I recommend these workflow examples: https://comfyanonymous.github.io/ComfyUI_examples/sdxl/

OPTIONS = {}

#@markdown **Models**

SDXL_1_0_BASE_AND_REFINER = True  #@param {type:"boolean"}
OPTIONS['SDXL_1_0_BASE_AND_REFINER'] = SDXL_1_0_BASE_AND_REFINER

if OPTIONS['SDXL_1_0_BASE_AND_REFINER']:
  !wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors -P ./models/checkpoints/
  !wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors -P ./models/checkpoints/

SD_2_0_MODELS = True  #@param {type:"boolean"}
OPTIONS['SD_2_0_MODELS'] = SD_2_0_MODELS

if OPTIONS['SD_2_0_MODELS']:
  !wget -c https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.safetensors -P ./models/checkpoints/
  !wget -c https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors -P ./models/checkpoints/

SD_1_5_MODEL = True  #@param {type:"boolean"}
OPTIONS['SD_1_5_MODEL'] = SD_1_5_MODEL

if OPTIONS['SD_1_5_MODEL']:
  !wget -c https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -P ./models/checkpoints/

#@markdown **VAEs**

SDXL_1_0_VAE = True  #@param {type:"boolean"}
OPTIONS['SDXL_1_0_VAE'] = SDXL_1_0_VAE

if OPTIONS['SDXL_1_0_VAE']:
  !wget -c https://civitai.com/api/download/models/183894 -O ./models/vae/sdxl-vae-fp16-fix.safetensors #sdxl-vae-fp16-fix.safetensors

SD_1_5_VAE = True  #@param {type:"boolean"}
OPTIONS['SD_1_5_VAE'] = SD_1_5_VAE

if OPTIONS['SD_1_5_VAE']:
  !wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -P ./models/vae/

#@markdown **T2I Adapters**

T2I_ADAPTER = True  #@param {type:"boolean"}
OPTIONS['T2I_ADAPTER'] = T2I_ADAPTER

if OPTIONS['T2I_ADAPTER']:
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_depth_sd14v1.pth -P ./models/controlnet/
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_seg_sd14v1.pth -P ./models/controlnet/
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd14v1.pth -P ./models/controlnet/
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_keypose_sd14v1.pth -P ./models/controlnet/
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_openpose_sd14v1.pth -P ./models/controlnet/
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -P ./models/controlnet/
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_canny_sd14v1.pth -P ./models/controlnet/

T2I_STYLES_MODEL = True  #@param {type:"boolean"}
OPTIONS['T2I_STYLES_MODEL'] = T2I_STYLES_MODEL

if OPTIONS['T2I_STYLES_MODEL']:
  !wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -P ./models/style_models/

#@markdown **Controlnets**

SDXL_1_0_CONTROLNETS = True  #@param {type:"boolean"}
OPTIONS['SDXL_1_0_CONTROLNETS'] = SDXL_1_0_CONTROLNETS

if OPTIONS['SDXL_1_0_CONTROLNETS']:
  !wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-canny-rank256.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-depth-rank256.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-recolor-rank256.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-sketch-rank256.safetensors -P ./models/controlnet/

SD_1_5_CONTROLNETS = True  #@param {type:"boolean"}
OPTIONS['SD_1_5_CONTROLNETS'] = SD_1_5_CONTROLNETS

if OPTIONS['SD_1_5_CONTROLNETS']:
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -P ./models/controlnet/
  !wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11u_sd15_tile_fp16.safetensors -P ./models/controlnet/

#@markdown **Upscalers**

ESRGAN_UPSCALER = True  #@param {type:"boolean"}
OPTIONS['ESRGAN_UPSCALER'] = ESRGAN_UPSCALER

if OPTIONS['ESRGAN_UPSCALER']:
  !wget -c https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P ./models/upscale_models/
  !wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth -P ./models/upscale_models/
  !wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth -P ./models/upscale_models/

#@markdown **AnimateDiff**

AD_MOTION_MODELS = True  #@param {type:"boolean"}
OPTIONS['AD_MOTION_MODELS'] = AD_MOTION_MODELS

if OPTIONS['AD_MOTION_MODELS']:
  !wget -c https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v14.ckpt -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/models/
  !wget -c https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15.ckpt -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/models/
  !wget -c https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15_v2.ckpt -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/models/

AD_MOTION_LORAS = True  #@param {type:"boolean"}
OPTIONS['AD_MOTION_LORAS'] = AD_MOTION_LORAS

if OPTIONS['AD_MOTION_LORAS']:
  !wget -c https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_PanLeft.ckpt -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora/
  !wget -c https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_PanRight.ckpt -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora/
  !wget -c https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_RollingAnticlockwise.ckpt -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora/
  !wget -c https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_RollingClockwise.ckpt -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora/
  !wget -c https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_TiltDown.ckpt -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora/
  !wget -c https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_TiltUp.ckpt -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora/
  !wget -c https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_ZoomIn.ckpt -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora/
  !wget -c https://huggingface.co/guoyww/animatediff/resolve/main/v2_lora_ZoomOut.ckpt -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora/


## Run ComfyUI


In [ ]:
#@markdown **Connecting to Cloudflare...**

import os
import sys
import shutil
import subprocess
import threading
import time
import socket
import urllib.request

# Detect if running inside Google Colab
in_colab = False
try:
    import google.colab  # type: ignore
    in_colab = True
except Exception:
    in_colab = False

# Global flag to prevent multiple cloudflared installations
_cloudflared_started = False

def try_install_cloudflared_deb(deb_url, deb_path):
    try:
        print(f"Downloading cloudflared from {deb_url} to {deb_path}...")
        urllib.request.urlretrieve(deb_url, deb_path)
        print("Download finished, installing .deb with dpkg...")
        subprocess.run(["dpkg", "-i", deb_path], check=True, stderr=subprocess.DEVNULL)
        print("cloudflared installed successfully (dpkg).")
        return True
    except Exception as e:
        print("Failed to install cloudflared via .deb:", e)
        return False


def iframe_thread(port):
    global _cloudflared_started

    # Prevent multiple runs
    if _cloudflared_started:
        return
    _cloudflared_started = True

    # Wait for local ComfyUI port to open
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
    print("\nComfyUI finished loading, attempting to launch cloudflared (if available)...\n")

    cf_exe = shutil.which('cloudflared')

    # If cloudflared not found, try to install it on Colab or Linux with dpkg
    if not cf_exe:
        if in_colab or sys.platform.startswith('linux'):
            if shutil.which('dpkg'):
                deb_url = 'https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb'
                # use /tmp in Colab/Linux
                deb_path = '/tmp/cloudflared-linux-amd64.deb' if os.path.isdir('/tmp') else os.path.join(os.getcwd(), 'cloudflared-linux-amd64.deb')
                ok = try_install_cloudflared_deb(deb_url, deb_path)
                if ok:
                    cf_exe = shutil.which('cloudflared')
            else:
                if in_colab:
                    print('dpkg not found in Colab environment — unexpected. Skipping automatic installation.')
                else:
                    print('dpkg not found: cannot install .deb automatically on this Linux system.')
        else:
            # Not a Linux/Colab environment
            print('cloudflared executable not found on this system. Tunneling will be skipped.')
            print('If you want remote access in Colab, open the notebook in Colab and run the cells there; on Windows, install cloudflared manually and run locally.')

    if cf_exe:
        try:
            p = subprocess.Popen([cf_exe, 'tunnel', '--url', f'http://127.0.0.1:{port}'],
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE,
                               text=True,
                               bufsize=1)
            # Parse stderr for the trycloudflare URL with timeout
            start_time = time.time()
            cf_url = None

            while time.time() - start_time < 30:  # 30 second timeout
                line = p.stderr.readline()
                if not line:
                    break
                if 'trycloudflare.com' in line:
                    # Extract URL from line
                    if 'http' in line:
                        cf_url = line[line.find('http'):].strip()
                        # Clean up the URL if there's extra text
                        if ' ' in cf_url:
                            cf_url = cf_url.split()[0]
                        break

            if cf_url:
                print('\n' + '='*60)
                print('🌐 ComfyUI Public URL:', cf_url)
                print('='*60 + '\n')
            else:
                print('⚠️  Cloudflared started but URL not found in output.')
                print('    Try accessing ComfyUI via local URL: http://127.0.0.1:8188')
        except Exception as e:
            print('Failed to start cloudflared:', e)
    else:
        print('cloudflared not available — running without public tunnel.')


# Start the thread that will wait for the server and try to create a tunnel
threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

# Finally, launch ComfyUI (run in the same process environment)
# On Windows this will simply start the server locally. On Colab/Linux it will behave the same.
print("Starting ComfyUI server...")
print("=" * 60)
sys.stdout.flush()

# Launch ComfyUI and capture output line by line for real-time display
process = subprocess.Popen(
    [sys.executable, 'main.py'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    bufsize=1,
    universal_newlines=True
)

# Print output in real-time
for line in process.stdout:
    print(line, end='')
    sys.stdout.flush()

# Wait for process to complete
exit_code = process.wait()
if exit_code != 0:
    print(f'\nComfyUI exited with code {exit_code}')
